In [1]:
import urllib

urllib.urlretrieve("http://www.digimouth.com/news/media/2011/09/google-logo.jpg", "local-filename.jpg")

('local-filename.jpg', <httplib.HTTPMessage instance at 0x7fb31815b050>)